# Predicting Rate my Professor Ratings

### Identifying the predictive Task 

**What is our predictive task?**

In this project we will analyze a dataset of user's rate my professor ratings and predict what rating a user will give to a certain professor. Our data set contains 5 features: professor name, review text, rating, difficulty and time/date the review was written.

**How will we evaluate our model?**

To evaluate this model, we will use MSE, MAE, and RMSE for each of our models based on the our predicted labels and the validation set. 

**What are our baselines?**

Relevant baselines to compare to our prediction models to would be a naive model which only predicts the global mean for each professor. We will calculate the global mean of the entire data set (ie.. every rating of every professor) for the baseline prediction.

**How do we know our model is valid?**

To ensure the validity of our model, we will compare the RMSE, MSE, and MAE of our model in comparison to the naive mean prediction to ensure that our model performs better than than the baseline. Additionally, we will train the model on a subset of the data set and allocate another portion as our test set to guarantee that our model works for data it has not yet been trained on which is a direct indicator of it's real world performance. 

In [5]:
!pip install datasets

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 9.9 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 10.1 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 9.2 MB/s  0:00:03m0:00:0100:01m
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 5.9 MB/s  0:00:01eta 0:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36/36 [datasets]/36 [datasets]ce-hub]]


In [1]:
from datasets import load_dataset

ds = load_dataset("ZephyrUtopia/ratemyprofessors_reviews")

/Users/luisxu/Documents/cse158/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

df_train = ds["train"].to_pandas()
df_test = ds["test"].to_pandas()

### Exploratory Analysis, Data Collection, Preprocessing, and Discussion

### Modeling

In [3]:
import numpy as np
def MSE(y, ypred):
    y = np.array(y, dtype=float)
    ypred = np.array(ypred, dtype=float)
    return np.mean((y - ypred) ** 2)

def getGlobalAverage(trainRatings):
    # Return the average rating in the training set
    return np.mean(trainRatings)



In [4]:
df_train

,name,text,rating,difficulty,date
0,Leila Hawana,"Although the homework is not too much, they ar...",4.0,5.0,2018-11-01
1,Gloria Magana,Where can I start she's a very tough grader. H...,1.0,5.0,2020-12-15
2,Teresa Canganelli,She is a wonderful professor! Very interactive...,5.0,4.0,2020-02-20
3,Kumar Ankit,Study solutions had incorrect answers taken fr...,1.0,4.0,2018-12-11
4,Irina Garces,Super nice and helpful. Was always there in of...,5.0,4.0,2019-05-15
...,...,...,...,...,...
336234,Ruth Chananie,"Great professor, weekly discussions. Great wri...",5.0,3.0,2021-12-12
336235,Jennifer Baggett,She is an amazing teacher and a great person t...,5.0,2.0,2021-05-20
336236,JoAnn Balingit,Really fun professor! Her class is super inter...,5.0,2.0,2018-10-31
336237,Jacobson Blomquist,This professor holds the textbook and reads ou...,1.0,4.0,2018-12-11


In [5]:
train_ratings = np.array(df_train['rating'], dtype='float')
test_ratings = np.array(df_test['rating'], dtype='float')
print(getGlobalAverage(train_ratings))

3.7674704600001783


In [6]:
print(MSE(train_ratings, getGlobalAverage(train_ratings)))

2.4709575975729883


In [7]:
!pip install scikit-learn

  Using cached scikit_learn-1.7.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
Using cached scikit_learn-1.7.2-cp312-cp312-macosx_12_0_arm64.whl (8.6 MB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [scikit-learn] [scikit-learn]


In [8]:
from sklearn.preprocessing import OneHotEncoder

df_train["timestamp"] = pd.to_datetime(df_train["date"])
df_train["year"] = df_train["timestamp"].dt.year
df_train["month"] = df_train["timestamp"].dt.month
df_train["dayofweek"] = df_train["timestamp"].dt.dayofweek
df_train["day"] = df_train["timestamp"].dt.day


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

text_vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1,2),
    stop_words="english"
)

In [24]:
from sklearn.compose import ColumnTransformer

X = df_train[["name", "difficulty", "year", "month", "day", "dayofweek", "text"]]
y = df_train["rating"]

preprocess = ColumnTransformer(
    transformers=[
        ("name", OneHotEncoder(handle_unknown="ignore"), ["name"]),
        ("num", "passthrough", ["difficulty", "year", "month", "day", "dayofweek"]),
        ("text", TfidfVectorizer(max_features=5000, stop_words="english"), "text"),
    ]
)

In [25]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline

model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("regressor", Ridge(alpha=1.0))
])

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model.fit(X_train, y_train)

,steps,"[('preprocess', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('name', ...), ('num', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [29]:
from sklearn.metrics import mean_squared_error

preds = model.predict(X_test)
preds = np.clip(preds, a_min=1, a_max=5)
mse = mean_squared_error(y_test, preds)
print("MSE:", mse)

MSE: 0.7365341664803989


In [32]:
df_test["timestamp"] = pd.to_datetime(df_test["date"])
df_test["year"] = df_test["timestamp"].dt.year
df_test["month"] = df_test["timestamp"].dt.month
df_test["dayofweek"] = df_test["timestamp"].dt.dayofweek
df_test["day"] = df_test["timestamp"].dt.day

In [33]:
X_valid = df_test[["name", "difficulty", "year", "month", "day", "dayofweek", "text"]]
y_valid = df_test["rating"]

preds = model.predict(X_valid)
preds = np.clip(preds, a_min=1, a_max=5)
mse = mean_squared_error(y_valid, preds)
print("MSE:", mse)

MSE: 0.7346041226216686


### Evaluation

### Discussion of Related Work